<a href="https://colab.research.google.com/github/uci-tuan4/Reddit-to-Instagram-Content-Pipeline/blob/main/osu_mapper_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This generates playable OSU beatmaps from just an mp3 file.

Credit goes to [OliBomby](https://github.com/OliBomby) for the osuT5 code and the pre-trained diffusion model.

**Instructions:**

1. Make sure you are using a GPU runtime type.
2. Run the "Setup" cell and wait for it to finish.
3. Upload an mp3 file using the file manager on the left side bar.
4. Run the "Fetch audio files" cell and select the file from the list.
5. Change the configuration settings as you like, then run the "Setup configuration" cell.
6. Run the "Generate" cell and wait for a few minutes.
7. After it's done, you will need to refresh the file manager by clicking on the refresh arrow icon. You will then see the .osz file; right-click on it to download.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [5]:
#@title Setup environment
#@markdown Setup
!git lfs install
!git clone https://huggingface.co/Tiger14n/osu_mapper
%cd /content/osu_mapper
!pip install -r requirements.txt

Updated git hooks.
Git LFS initialized.
Cloning into 'osu_mapper'...
remote: Enumerating objects: 201, done.
remote: Total 201 (delta 0), reused 0 (delta 0), pack-reused 201 (from 1)
Receiving objects: 100% (201/201), 335.99 KiB | 19.76 MiB/s, done.
Resolving deltas: 100% (41/41), done.
Filtering content: 100% (4/4), 2.89 GiB | 55.61 MiB/s, done.
/content/osu_mapper


In [6]:
#@title Fetch audio files
import os
import mimetypes
import ipywidgets as widgets
from IPython.display import display

def find_audio_files(directory):
    audio_files = []

    # Ensure mimetype database is initialized
    mimetypes.init()

    # Walk through the directory
    for root, dirs, files in os.walk(directory):
        for file in files:
            # Get the full file path
            file_path = os.path.join(root, file)

            # Get the MIME type of the file
            mime_type, _ = mimetypes.guess_type(file_path)

            # Check if it's an audio file
            if mime_type and mime_type.startswith('audio'):
                audio_files.append(file_path)
    if len(audio_files) == 0:
        print("No audio files found, Please upload an audio file using file manager on the left side of the page")

    return audio_files

# Example usage
directory_path = '/content'
audio_file_list = find_audio_files(directory_path)

if audio_file_list:
  audio_file = widgets.Dropdown(options=audio_file_list, value=audio_file_list[0])
  print("Select an audio file from the list:")
  display(audio_file)

Select an audio file from the list:


Dropdown(options=('/content/Kill That Girl.mp3',), value='/content/Kill That Girl.mp3')

# Generate one beatmap



In [ ]:
%cd /content/osu_mapper
#@title Setup configuration

# @markdown Target difficulty: (The model can only generate 3* to 7*):
target_difficulty = 6 # @param {type:"number"}
# @markdown  Info:
#@title Configuration
song_title = "Kill That Girl" # @param {type:"string"}
song_artist = "Maximum Utility" # @param {type:"string"}
map_creator = "OSU Anonymous" # @param {type:"string"}
# @markdown BPM of the selcted song (Optional)
bpm = 93 # @param {type:"number"}
# @markdown The offset to first beat in milliseconds (Optional)
offset = 0 # @param {type:"number"}
# @markdown resnap hit-objects is recomeneded but requries accurate BPM and offset for songs with consistent BPM, it's expermintal and doesn't always work (Optional)
resnap_hitobjects = False  # @param {type:"boolean"}
#  @markdown #Diffusion settings:
style = "Sotarks" #@param["None", "Cheri", "Monstrata", "Sotarks", "wafer", "Nevo", "ScubDomino", "Night Mare"]

num_sampling_steps = "250" #@param[100,250,500,1000]

style_to_id = {
    "None": None,
    "Cheri": 2592760,
    "Monstrata": 1451282,
    "Sotarks": 1995061,
    "wafer": 3697057,
    "Nevo": 2799753,
    "ScubDomino": 1772923,
    "Night Mare": 1907310,
}


style_id = style_to_id[style]


full_set = False

# update the config file
import yaml
# Load the existing YAML file
with open('configs/inference.yaml', 'r') as file:
    config = yaml.safe_load(file)


# Update the configuration values
config['audio_path'] = audio_file.value
config['title'] = song_title
config['artist'] = song_artist
config['creator'] = map_creator
config['bpm'] = bpm
config['offset'] = offset
config['full_set'] = full_set
config['snap_hitobjects'] = resnap_hitobjects
config['difficulty'] = target_difficulty
config['diffusion']['style_id'] = style_id
config['diffusion']['num_sampling_steps'] = num_sampling_steps
config['output_path'] = '/content'


# Save the updated configuration back to the YAML file
with open('configs/inference.yaml', 'w') as file:
    yaml.dump(config, file)


print("Configuration updated successfully!, Run the generate cell")

In [9]:
#@title Generate
!python3 inference.py

2025-04-14 00:41:56.212292: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744591316.235385    5058 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744591316.243652    5058 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-14 00:41:56.264929: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Error executing job with overrides: []
Traceback (most recent call last):
  File "/content/osu_mapper/inference.py", 

# Generate a full mapset (Optional)

In [3]:
%cd /content/osu_mapper
#@title Setup configuration

# @markdown **This generates a full mapset of 5 beatmaps ranging from 3 to 7 stars, this can take up to 15 minutes**

# @markdown  Info:
#@title Configuration
song_title = "OSU" # @param {type:"string"}
song_artist = "OSU Mapper" # @param {type:"string"}
map_creator = "OSU Mapper" # @param {type:"string"}
# @markdown BPM of the selcted song (Optional)
bpm = 120 # @param {type:"number"}
# @markdown The offset to first beat in milliseconds (Optional)
offset = 0 # @param {type:"number"}
# @markdown resnap hit-objects is recomeneded but requries accurate BPM and offset for songs with consistent BPM, it's expermintal and doesn't always work (Optional)
resnap_hitobjects = False  # @param {type:"boolean"}
#  @markdown #Diffusion settings:
style = "Monstrata" #@param["None", "Cheri", "Monstrata", "Sotarks", "wafer", "Nevo", "ScubDomino", "Night Mare"]

num_sampling_steps = "100" #@param[100,250,500,1000]

style_to_id = {
    "None": None,
    "Cheri": 2592760,
    "Monstrata": 1451282,
    "Sotarks": 1995061,
    "wafer": 3697057,
    "Nevo": 2799753,
    "ScubDomino": 1772923,
    "Night Mare": 1907310,
}


style_id = style_to_id[style]


full_set = True

# update the config file
import yaml
# Load the existing YAML file
with open('configs/inference.yaml', 'r') as file:
    config = yaml.safe_load(file)


# Update the configuration values
config['audio_path'] = audio_file.value
config['title'] = song_title
config['artist'] = song_artist
config['creator'] = map_creator
config['bpm'] = bpm
config['offset'] = offset
config['full_set'] = full_set
config['snap_hitobjects'] = resnap_hitobjects
config['diffusion']['style_id'] = style_id
config['diffusion']['num_sampling_steps'] = num_sampling_steps
config['output_path'] = '/content'


# Save the updated configuration back to the YAML file
with open('configs/inference.yaml', 'w') as file:
    yaml.dump(config, file)


print("Configuration updated successfully!, Run the generate cell")

/content/osu_mapper
Configuration updated successfully!, Run the generate cell


In [4]:
#@title Generate
!python3 inference.py

Traceback (most recent call last):
  File "/content/osu_mapper/inference.py", line 9, in <module>
    from osuT5.inference import Preprocessor, Pipeline, Postprocessor, DiffisionPipeline
  File "/content/osu_mapper/osuT5/inference/__init__.py", line 1, in <module>
    from .pipeline import *
  File "/content/osu_mapper/osuT5/inference/pipeline.py", line 15, in <module>
    from osuT5.model import OsuT
  File "/content/osu_mapper/osuT5/model/__init__.py", line 1, in <module>
    from .osu_t import OsuT
  File "/content/osu_mapper/osuT5/model/osu_t.py", line 8, in <module>
    from transformers import T5Config, T5ForConditionalGeneration, WhisperForConditionalGeneration, WhisperConfig
  File "<frozen importlib._bootstrap>", line 1229, in _handle_fromlist
  File "/usr/local/lib/python3.11/dist-packages/transformers/utils/import_utils.py", line 1965, in __getattr__
    value = getattr(module, name)
            ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/transforme